In [1]:
import copy
import os
import subprocess
import time
import tempfile
import sys

from pathlib import Path
from numpy.linalg import inv, norm, pinv, svd,eig
from scipy.optimize import fmin_bfgs, fmin_slsqp, minimize
from yourdfpy import URDF
from pinocchio.visualize import MeshcatVisualizer

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pinocchio as pin

# Joint Limits
Define the joint configuration (rad) and velocity (rad/s) limits for the Kinova Gen 2 arm

In [2]:
# Define joint limits
joint_to_limits = {
    "robot_tilt": [-np.pi/2, np.pi/2],
    "j2n6s200_joint_1": [-np.pi, np.pi],
    "j2n6s200_joint_2": [-np.pi, np.pi],
    "j2n6s200_joint_3": [-np.pi, np.pi],
    "j2n6s200_joint_4": [-np.pi, np.pi],
    "j2n6s200_joint_5": [-np.pi, np.pi],
    "j2n6s200_joint_6": [-np.pi, np.pi],
    "j2n6s200_joint_finger_1": [0, 0],
    "j2n6s200_joint_finger_2": [0, 0],
}

# Define joint velocity limits from https://github.com/personalrobotics/ada_ros2/blob/1ca71e2b5a7ffaff43dcb5a6fe90114c8d5ac335/ada_moveit/config/joint_limits.yaml
joint_velocity_limits = np.array([
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.15, # robot_tilt
    0.62831853071795862, # j2n6s200_joint_1
    0.62831853071795862, # j2n6s200_joint_2
    0.62831853071795862, # j2n6s200_joint_3
    0.83775804095727813, # j2n6s200_joint_4
    0.83775804095727813, # j2n6s200_joint_5
    0.83775804095727813, # j2n6s200_joint_6
    1.0, # j2n6s200_joint_finger_1
    1.0, # j2n6s200_joint_finger_2
])

# Sample configurations
Sample joint configurations by either sampling `num_samples` evenly for each joint within their joint configuration limits, or sampling randomly from a uniform distribution

In [3]:
# Sample joint positions evenly
num_samples = 4
joint_positions = [np.linspace(limits[0], limits[1], num_samples) for limits in joint_to_limits.values()]

# Create a meshgrid of all combinations
meshgrid = np.meshgrid(*joint_positions)

# Generate the list of configurations to evaluate from the meshgrid
configurations = np.array(meshgrid).reshape(meshgrid[0].size, -1)
print(len(configurations))

262144


In [3]:
def generate_random_joint_positions(joint_to_limits, num_samples):

  joint_positions_list = []
  for _ in range(num_samples):
    joint_positions = np.zeros(len(joint_to_limits))
    for i, (joint_name, limits) in enumerate(joint_to_limits.items()):
      joint_positions[i] = np.random.uniform(limits[0], limits[1])
    joint_positions_list.append(joint_positions)
  return joint_positions_list

num_samples = 10000
rand_configurations = generate_random_joint_positions(joint_to_limits, num_samples)

# Load URDF model of ADA
Build a Pinocchio model using URDF for ADA and initialize a MeshCat visualizer

In [35]:
cwd = Path(os.getcwd())
urdf_file_path = cwd / "ada_no_forque.urdf"
mesh_dir = cwd.parent / "ada_description/meshes/"
ros_url_prefix = "package://"
abs_path_prefix = str(cwd.parent) + "/"

with tempfile.TemporaryDirectory() as tmpdirname:
    tmp_file_path = Path(tmpdirname)/urdf_file_path.name

    with open(urdf_file_path, 'r') as fin:
        with open(tmp_file_path, 'w') as fout:
            for line in fin:
                fout.write(line.replace(ros_url_prefix, abs_path_prefix))

    model, collision_model, visual_model = pin.buildModelsFromUrdf(str(tmp_file_path), str(mesh_dir), pin.JointModelFreeFlyer())

In [36]:
data = model.createData()
end_effector_joint_id = model.getJointId("j2n6s200_joint_6")

In [37]:
viz = MeshcatVisualizer(model, collision_model, visual_model)

# Initialize the viewer.
try:
    viz.initViewer(open=True)
except ImportError as error:
    print(error)
    sys.exit(0)

try:
    viz.loadViewerModel()
except AttributeError as error:
    print(error)
    sys.exit(0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7008/static/


# Compute maximum angular velocity
For each configuration, we find the optimal joint velocities `q_dot` within the joint velocity limits that maximizes the angular velocity of the end-effector. Across all configurations, we choose the global maximum angular velocity of the end-effector

In [38]:
def config_to_q(config):
    return np.array([
        0, # Base offset x
        0, # Base offset y
        0, # Base offset z
        0, # scaling
        0, # scaling
        0, # scaling
        1, # Default
        config[0], # robot_tilt
        np.cos(config[1]), # j2n6s200_joint_1 (cos)
        np.sin(config[1]), # j2n6s200_joint_1 (sin)
        config[2], # j2n6s200_joint_2
        config[3], # j2n6s200_joint_3
        np.cos(config[4]), # j2n6s200_joint_4 (cos)
        np.sin(config[4]), # j2n6s200_joint_4 (sin)
        np.cos(config[5]), # j2n6s200_joint_5 (cos)
        np.sin(config[5]), # j2n6s200_joint_5 (sin)
        np.cos(config[6]), # j2n6s200_joint_6 (cos)
        np.sin(config[6]), # j2n6s200_joint_6 (sin)
        config[7], # j2n6s200_joint_finger_1
        config[8], # j2n6s200_joint_finger_2
    ])

In [39]:
from multiprocessing.sharedctypes import Value

# Create a shared counter
counter = Value('i', 0)
def calculate_max_omega(config, index):
    q = config_to_q(config)
    
    # Compute Jacobian for current configuration
    pin.forwardKinematics(model, data, q)
    J = pin.computeJointJacobian(model, data, q, end_effector_joint_id)

    # Define the objective function
    def objective(q_dot):
        omega = J[:3, :] @ q_dot
        return -np.linalg.norm(omega)
    
    # Define the constraints
    constraints = ({'type': 'ineq', 'fun': lambda x: joint_velocity_limits - x},
                   {'type': 'ineq', 'fun': lambda x: x + joint_velocity_limits})
    
    # Initial guess for joint velocities
    q_dot_init = np.zeros(model.nv)

    # Solve the optimization problem
    result = minimize(objective, q_dot_init, method='SLSQP', constraints=constraints)

    # Optimal joint velocities
    optimal_q_dot = result.x

    # Calculate the maximum angular velocity
    max_omega = np.linalg.norm(J[:3, :] @ optimal_q_dot)

    # Update the shared counter
    with counter.get_lock():
        counter.value += 1
        print(f"\rCompleted configuration {counter.value}/{len(rand_configurations)}", end="")

    return max_omega, config

In [40]:
from joblib import Parallel, delayed

# Parallelize the loop over configurations
with Parallel(n_jobs=-1, backend="threading") as parallel:
    max_angular_velocities_with_configs = parallel(delayed(calculate_max_omega)(config, i) for i, config in enumerate(rand_configurations))


Completed configuration 10000/10000

In [41]:
# Find the maximum angular velocity across all configurations
max_angular_velocity_across_configs = max(max_angular_velocities_with_configs, key=lambda x: x[0])
print("\nMaximum angular velocity across all configurations:", max_angular_velocity_across_configs[0])
print("Associated configuration:", max_angular_velocity_across_configs[1])

viz.display(config_to_q(max_angular_velocity_across_configs[1]))


Maximum angular velocity across all configurations: 1.0192929711753709
Associated configuration: [-0.80370985  1.36499043 -1.55476502  3.08961834  0.62685185  0.03867948
 -2.90832305  0.          0.        ]
